## Basic example

In [66]:
import pop2net as p2n

# create environment
env = p2n.Environment()

# create 3 actors & add them to the environment
actor1 = p2n.Actor()
actor2 = p2n.Actor()
actor3 = p2n.Actor()
env.add_actors([actor1, actor2, actor3])

# Define (empty) location classes
class Friendship(p2n.Location):
    pass

class School(p2n.Location):
    pass

# create school location, add it to the environment & add all actors to the school
school = School()
env.add_location(school)
school.add_actors(env.actors)

# connect two actors via a friendship location
actor1.connect(actor2, location_cls=Friendship)

# get all actors that are connected with actor1 via friendship
actor1.neighbors(location_labels=["Friendship"])

In [67]:
# create a NetworkInspector and plot the bipartite network
inspector = p2n.NetworkInspector(env)
inspector.plot_bipartite_network()

## Example with creator

In [68]:
import pandas as pd

data = [
    {"household_id": 1, "industry": "cars", "work_hours": 8},
    {"household_id": 2, "industry": "food", "work_hours": 4},
    {"household_id": 2, "industry": "cars", "work_hours": 6},
    {"household_id": 2, "industry": "food", "work_hours": 8},
    {"household_id": 3, "industry": "cars", "work_hours": 16},
    {"household_id": 3, "industry": "food", "work_hours": 12},
    {"household_id": 3, "industry": "food", "work_hours": 0},
    {"household_id": 3, "industry": "food", "work_hours": 0},

]
df = pd.DataFrame(data)
df

,household_id,industry,work_hours
0,1,cars,8
1,2,food,4
2,2,cars,6
3,2,food,8
4,3,cars,16
5,3,food,12
6,3,food,0
7,3,food,0


In [69]:
import pop2net as p2n

env = p2n.Environment()
creator = p2n.Creator(env)
inspector = p2n.NetworkInspector(env)


# design city locations
class City(p2n.LocationDesigner):
    n_locations = 2 # set the number of cities to 2


# design households
class Household(p2n.LocationDesigner):
    def split(self, actor):
        """Create one household location for each household id."""
        return actor.household_id

    def weight(self, actor):
        """Set the weight between the actor and the location."""
        return 24 - actor.work_hours


# design work places
class Work(p2n.LocationDesigner):
    n_actors = 5 # set the number of actors per work location to 5

    def split(self, actor):
        """Create seperated work locations for each industry."""
        return actor.industry

    def weight(self, actor):
        """Set the weight between the actor and the location."""
        return actor.work_hours
    
    def filter(self, actor):
        """Only assign actors with more than 0 work hours to this location type."""
        return actor.work_hours > 0
    

# create the agents and connect them
creator.create(
    df=df, # sample from this data to create the actors
    n_actors=100, # set the number of actors to 100
    sample_level="household_id", # sample whole households
    location_designers=[City, Household, Work], # connect the actors via cities, households and work places
    )

# plot bipartite and unipartite networks
inspector.plot_networks(location_color="label")